In [1]:
import pandas as pd

In [4]:
train = pd.read_csv("twitter/Corona_NLP_train.csv",encoding='latin1')
test = pd.read_csv("twitter/Corona_NLP_test.csv",encoding='latin1')
train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [8]:
train['Sentiment'].value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

# Preprocessing

In [9]:
# transform sentiment to int
def senti_to_int(s):
    if s == 'Extremely Negative':
        return 0
    elif s == 'Negative':
        return 1
    elif s == 'Neutral':
        return 2
    elif s == 'Positive':
        return 3
    elif s == 'Extremely Positive':
        return 4
    
y_train = [senti_to_int(s) for s in train['Sentiment']]
y_test = [senti_to_int(s) for s in test['Sentiment']]

In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/F970614/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# load stop words
from nltk.corpus import stopwords
stop_word = stopwords.words('english')

In [20]:
import re 
def clean(text):
    #remove urls
    text = re.sub(r'http\S+', " ", text)
    #remove mentions
    text = re.sub(r'@\w+',' ',text)
    #remove hastags
    text = re.sub(r'#\w+', ' ', text)
    #remove digits
    text = re.sub(r'\d+', ' ', text)
    #remove html tags
    text = re.sub('r<.*?>',' ', text) 
    #remove stop words 
    text = text.split()
    text = " ".join([word for word in text if not word in stop_word])    
    return text

In [25]:
print(train['OriginalTweet'][1])
print(clean(train['OriginalTweet'][1]))

advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order
advice Talk neighbours family exchange phone numbers create contact list phone numbers neighbours schools employer chemist GP set online shopping accounts poss adequate supplies regular meds order


In [26]:
x_train = [clean(x) for x in train['OriginalTweet']]
x_test = [clean(x) for x in test['OriginalTweet']]

In [30]:
# transform text to numeric vector
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [32]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(x_train)
sequences_train = tokenizer.texts_to_sequences(x_train)
sequences_test = tokenizer.texts_to_sequences(x_test)

In [47]:
v=len(tokenizer.word_index)

In [36]:
print(x_train[1])
print(sequences_train[1])

advice Talk neighbours family exchange phone numbers create contact list phone numbers neighbours schools employer chemist GP set online shopping accounts poss adequate supplies regular meds order
[439, 819, 2512, 199, 2632, 801, 1178, 1029, 362, 320, 801, 1178, 2512, 1161, 3509, 3248, 4522, 418, 13, 12, 2251, 3037, 118, 989, 2894, 165]


In [41]:
x_train = pad_sequences(sequences_train)
x_train.shape

(41157, 53)

In [43]:
x_test = pad_sequences(sequences_test, maxlen = 53)
x_test.shape

(3798, 53)

In [51]:
import tensorflow as tf

In [53]:
x_train = tf.cast(x_train,tf.int32)
x_test = tf.cast(x_test,tf.int32)
y_train = tf.cast(y_train,tf.int32)
y_test = tf.cast(y_test,tf.int32)

# RNN

In [44]:
from tensorflow import keras
from keras import layers

In [74]:
embeddingdim = 32
model=keras.Sequential(
        [   layers.InputLayer(input_shape=(41157,)),
            layers.Embedding(v+1, embeddingdim),
            layers.SimpleRNN(units = 64, activation = 'relu', return_sequences = True),
            layers.GlobalMaxPool1D(),
            layers.Dense(32,activation='relu'),
            layers.Dense(5, activation="softmax")
        ])
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [75]:
history = model.fit(x_train, y_train, epochs=4)

Epoch 1/4
1287/1287 [==============================] - 35s 26ms/step - loss: 1.0329 - accuracy: 0.5809
Epoch 2/4
1287/1287 [==============================] - 35s 27ms/step - loss: 0.6040 - accuracy: 0.7926
Epoch 3/4
1287/1287 [==============================] - 33s 25ms/step - loss: 0.4903 - accuracy: 0.8396
Epoch 4/4
1287/1287 [==============================] - 34s 27ms/step - loss: 0.4059 - accuracy: 0.8676


In [76]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(test_acc)

119/119 - 1s - loss: 0.7533 - accuracy: 0.7420
0.7419694662094116


In [78]:
y_predict = model.predict(x_test)

In [92]:
import numpy as np

In [93]:
np.argmax(y_predict, axis=1, out=None)

array([0, 3, 4, ..., 2, 0, 4])

In [94]:
df = pd.DataFrame({'predict':np.argmax(y_predict, axis=1, out=None),
            "true":y_test,
            "text":test['OriginalTweet']})

In [95]:
df.head()

,predict,true,text
0,0,0,TRENDING: New Yorkers encounter empty supermar...
1,3,3,When I couldn't find hand sanitizer at Fred Me...
2,4,4,Find out how you can protect yourself and love...
3,1,1,#Panic buying hits #NewYork City as anxious sh...
4,2,2,#toiletpaper #dunnypaper #coronavirus #coronav...


# LSTM

In [99]:
embeddingdim = 32
model1=keras.Sequential(
        [   layers.InputLayer(input_shape=(41157,)),
            layers.Embedding(v+1, embeddingdim),
            layers.LSTM(units = 64, activation = 'relu', return_sequences = True),
            layers.GlobalMaxPool1D(),
            layers.Dense(32,activation='relu'),
            layers.Dense(5, activation="softmax")
        ])
model1.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [100]:
history = model1.fit(x_train, y_train, epochs=4, validation_data=(x_test, y_test))

Epoch 1/4
1287/1287 [==============================] - 66s 50ms/step - loss: 1.3510 - accuracy: 0.4742 - val_loss: 0.9721 - val_accuracy: 0.6145
Epoch 2/4
1287/1287 [==============================] - 56s 44ms/step - loss: 0.7805 - accuracy: 0.7374 - val_loss: 0.7989 - val_accuracy: 0.7088
Epoch 3/4
1287/1287 [==============================] - 55s 43ms/step - loss: 0.5801 - accuracy: 0.8066 - val_loss: 0.7910 - val_accuracy: 0.7167
Epoch 4/4
1287/1287 [==============================] - 1150s 894ms/step - loss: 0.4868 - accuracy: 0.8411 - val_loss: 0.7807 - val_accuracy: 0.7401


In [101]:
test_loss, test_acc = model1.evaluate(x_test, y_test, verbose=2)
print(test_acc)

119/119 - 1s - loss: 0.7807 - accuracy: 0.7401
0.740126371383667


# Transformer

In [102]:
# encoder
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
    ## rate: drop out rate
    ## embed dim: d_k in multihead attention 
    ## ff_dim: hidden layer size in feed forward network 
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        # multihead attention layer+drop out
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        # residual connection+normalization
        out1 = self.layernorm1(inputs + attn_output)       
        # feed forward+drop out
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        # residual connection+normalization
        return self.layernorm2(out1 + ffn_output)
    
# embedding+position    
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [103]:
x_train.shape

TensorShape([41157, 53])

In [114]:
embed_dim = 32 
num_heads = 6
ff_dim = 32  
maxlen = 53
inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, v+1, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(5, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [115]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=3, validation_data=(x_test, y_test)
)

Epoch 1/3
1287/1287 [==============================] - 84s 64ms/step - loss: 0.9565 - accuracy: 0.6135 - val_loss: 0.7450 - val_accuracy: 0.7314
Epoch 2/3
1287/1287 [==============================] - 1164s 905ms/step - loss: 0.5649 - accuracy: 0.8001 - val_loss: 0.6878 - val_accuracy: 0.7483
Epoch 3/3
1287/1287 [==============================] - 81s 63ms/step - loss: 0.4815 - accuracy: 0.8367 - val_loss: 0.6887 - val_accuracy: 0.7583


# Conclusion

- Transformer seems to be better, but not to much. But it runs faster (great!)

- RNN and LSTM aren't too different, RNN even seems to be slightly better.

- Possible reason: sequence isn't that long, max_len = 53 in this case. Also, transformer are designed for seq2seq, here we drop the decoder layer to perform classification